# Import Packages

In [ ]:
import numpy as np
from multiprocessing import Pool

from LBL_funcs_fullSpectrum import *
from LBL_funcs_longwave import *

import warnings
warnings.filterwarnings('ignore')

# Define User specified input varibles

In [ ]:
def goes_nu(dnu=3):
    # Define the spectral wavenumber data of GOES
    #dnu = 3
    # Define the spectral wavenumber data of GOES
    min_max_values = {
        "C06": (4255.00, 4762.00),
        "C05": (6097.00, 6330.00),
        "C04": (7194.00, 7353.00),
        "C03": (11111.00, 13334.00),
        "C02": (13404.00, 18182.00),
        "C01": (20408.00, 22223.00),
    }
    arrays = []
    for key, (min_val, max_val) in min_max_values.items():
        array = np.arange(min_val, max_val, dnu)
        arrays.append(array)
    combined_array = np.concatenate(arrays)
    nu = combined_array
    return nu

In [ ]:
## general inputs
N_layer=18 # the number of atmospheric layers
dnu = 0.1 # spectral resolution 0.1 is enough, 0.01 is too fine, especially for cloudy periods
nu= goes_nu(dnu) # spectral grid on wavenumber
molecules=['H2O','CO2','O3','N2O','CH4','O2','N2'] # considered atmospheric gases
#current trace gas surface vmr from http://cdiac.ornl.gov/pns/current_ghg.html, except O3
vmr0={'H2O':0.03,'CO2':399.5/10**6,'O3':50/10**9,'N2O':328/10**9,
          'CH4':1834/10**9,'O2':2.09/10,'N2':7.81/10}
model='AFGL midlatitude summer' #profile model, 'AFGL tropical','AFGL midlatitude summer','AFGL midlatitude winter',
#'AFGL subarctic summer','AFGL subarctic winter','AFGL US standard'
cld_model = 'default' # cloud model, 'default' or 'caseX'
period = 'day' # choose 'day' or 'night' for proper temperature profile
spectral='LW' # choose 'LW' or 'SW'
inputs={'N_layer':N_layer, 'nu':nu, 'molecules':molecules,'vmr0':vmr0,
       'model':model,'cld_model':cld_model,'period':period,'spectral':spectral}


## inputs for desired atmoshperic conditions
rh0_v=np.arange(70,80,5)/100 # surface relative humidity
T_surf_v=np.arange(290,300,5) # surface temperature
AOD_v=np.array([0.05,0.1]) # aerosol optical depth at 479.5 nm
COD_v=np.array([0.0,5.0]) # cloud optical depth at 479.5 nm
#CODs=np.array([0,0.1,0.3,0.5,0.7,1.0,3.0,5.0,10.0,50.0]) # cloud optical depth at 479.5 nm
kap_v=[[4,5,6],[10,11,12]]
#kap_v=[[10],[8,9,10],[6,7,8,9,10],[4,5,6,7,8,9,10],
#      [22],[19,20,21,22],[16,17,18,19,20,21,22],[13,14,15,16,17,18,19,20,21,22],
#      [10,11,12,13,14,15,16,17,18,19,20,21,22],[7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
#      [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]] # cloud residancy layer


# spectral surface absorptance
alpha_s=np.zeros(len(nu))+1.0 # by default, the ground is black
# -----passive cooling material------
#temp=np.genfromtxt('Profile data/emissivity of passive cooling material.csv', delimiter=',')
#alpha_s=np.interp(nu,temp[:,1],temp[:,2])  


# folder directory to store the results
file_dir='results_longwave/tests/' # create the directory first

# (optional) Save spectral optical properties of gases to local directory

In [ ]:
# get spectral absorption coefficients of gases of all layers and save to local directory
# takes a couple minutes to run, require parallel computing
# temperature dependence of kappa_a is not strong, so this cell only needed to run once for different T_surf
# need to re-run if N_layer, model, period, molecules or nu changed
from LBL_funcs_getHitran import *
p,pa=set_pressure(N_layer) 
# use a representative surface temperature of 290 K for optical properties
t,ta=set_temperature(model,p,pa,290, period) 
inputs_kappa={'N_layer':N_layer,'model':model,'molecules':molecules,'nu':nu,
              'pa':pa,'ta':ta,'spectral':spectral}
getKappa_AllLayers(inputs_kappa)

# Run the LBL longwave model for different atmospheric conditions

In [ ]:
# parallel computing
for iT in range(0,len(T_surf_v)):
    for iAOD in range(0,len(AOD_v)):
        for iCOD in range(0,len(COD_v)):
            for iKAP in range(0,len(kap_v)):
                list_args=[]
                for iRH in range(0,len(rh0_v)):
                    properties={'rh0':rh0_v[iRH],'T_surf':T_surf_v[iT],'AOD':AOD_v[iAOD],
                                'COD':COD_v[iCOD],'kap':kap_v[iKAP]}
                    args={'properties':properties,'inputs':inputs,'alpha_s':alpha_s}
                    list_args.append(args)
                if __name__ == '__main__':
                    pool = Pool() # workers
                    results = list (pool.map(LBL_longwave,list_args))
                    pool.terminate()
                # save the results to local directory
                for iRH in range(0,len(rh0_v)):
                    outputs=results[iRH]
                    fileName="Results_{}layers_{}_RH={}_Tsurf={}_AOD={}_COD={}_kap={}.npy".format(
                        N_layer, model,rh0_v[iRH],T_surf_v[iT],AOD_v[iAOD],COD_v[iCOD],kap_v[iKAP])
                    np.save(file_dir+fileName,outputs)          

# Calculate Brunt's DLW for quick validation - only for clear skies when COD=0

In [ ]:
p,pa=set_pressure(N_layer)
z,za=set_height(model,p,pa)
t,ta=set_temperature(model,p,pa,T_surf_v[0], period)

# sample codes to access the modeling results
F_dw_v=np.zeros(len(rh0_v))
for iRH in range (0,len(rh0_v)):
    data=np.load(file_dir+"Results_{}layers_{}_RH={}_Tsurf={}_AOD={}_COD={}_kap={}.npy".format(
                        N_layer, model,rh0_v[iRH],T_surf_v[0],AOD_v[0],COD_v[0],kap_v[0]))
    outputs=data.item()
    F_dw=outputs['F_dw']
    F_dw_v[iRH]=F_dw[1]
print(F_dw_v)

# Brunt results
ps=saturation_pressure(t)
pw=ps[1]*rh0_v
Brunt=0.598+0.057*np.sqrt(pw/100)
sigma=5.667*10**(-8)
Brunt*=sigma*t[1]**4
print (Brunt)

# print the difference
print (F_dw_v-Brunt)